In [172]:
# Configuración inicial y librerías
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime
from pathlib import Path
from scipy.optimize import minimize
from scipy import stats
import requests
import warnings
warnings.filterwarnings('ignore')

# Configuración
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:.4f}' if pd.notna(x) else '')
np.random.seed(42)

print("✅ Configuración completada")
print(f"📊 Sistema listo para análisis financiero")
print(f"🌎 Librerías para datos macroeconómicos disponibles")

✅ Configuración completada
📊 Sistema listo para análisis financiero
🌎 Librerías para datos macroeconómicos disponibles


## 1. Carga y Preparación de Datos

Carga de datos históricos desde Excel y complemento con APIs financieras para crear dataset completo.

In [173]:
# CARGA DE DATOS COMPLETA Y LIMPIA
def obtener_dolar_ccl(fecha_inicio, fecha_fin):
    """Obtiene datos históricos completos del dólar CCL desde ArgentinaDatos"""
    try:
        print("🔄 Obteniendo datos históricos del Dólar CCL (ArgentinaDatos)...")
        
        # API de ArgentinaDatos con histórico completo
        url = "https://api.argentinadatos.com/v1/cotizaciones/dolares/contadoconliqui"
        response = requests.get(url, timeout=30)
        
        if response.status_code == 200:
            data = response.json()
            
            # Convertir a DataFrame
            df_ccl = pd.DataFrame(data)
            df_ccl['fecha'] = pd.to_datetime(df_ccl['fecha']).dt.date
            
            # Filtrar por el rango de fechas necesario
            fecha_inicio_date = fecha_inicio.date() if hasattr(fecha_inicio, 'date') else fecha_inicio
            fecha_fin_date = fecha_fin.date() if hasattr(fecha_fin, 'date') else fecha_fin
            
            df_filtrado = df_ccl[
                (df_ccl['fecha'] >= fecha_inicio_date) & 
                (df_ccl['fecha'] <= fecha_fin_date)
            ].copy()
            
            if len(df_filtrado) > 0:
                # Preparar datos finales
                df_filtrado['Fecha'] = pd.to_datetime(df_filtrado['fecha'])
                df_filtrado['Tipo_Cambio_ARSUSD'] = df_filtrado['venta']  # Usar precio de venta
                
                # Calcular retorno FX día a día
                df_filtrado = df_filtrado.sort_values('Fecha')
                df_filtrado['Retorno_FX'] = df_filtrado['Tipo_Cambio_ARSUSD'].pct_change()
                
                resultado = df_filtrado[['Fecha', 'Tipo_Cambio_ARSUSD', 'Retorno_FX']]
                
                print(f"✅ CCL histórico obtenido: {len(resultado)} registros")
                print(f"   Período: {resultado['Fecha'].min().strftime('%Y-%m-%d')} → {resultado['Fecha'].max().strftime('%Y-%m-%d')}")
                print(f"   Rango CCL: ${resultado['Tipo_Cambio_ARSUSD'].min():.1f} - ${resultado['Tipo_Cambio_ARSUSD'].max():.1f}")
                
                return resultado
            else:
                print("❌ No hay datos CCL para el período solicitado")
                return None
                
        else:
            print(f"❌ Error en API ArgentinaDatos: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"❌ Error obteniendo CCL histórico: {str(e)}")
        return None

def obtener_tasa_plazo_fijo_historica(fecha_inicio, fecha_fin):
    """Obtiene tasas históricas de plazo fijo a 30 días desde API del BCRA"""
    try:
        print("🔄 Obteniendo tasas históricas de plazo fijo del BCRA...")
        
        # API del BCRA v3.0 - Tasas de depósitos a 30 días (idVariable=12)
        url = "https://api.bcra.gob.ar/estadisticas/v3.0/Monetarias/12"
        params = {
            "desde": fecha_inicio.strftime('%Y-%m-%d'),  
            "hasta": fecha_fin.strftime('%Y-%m-%d')
        }
        
        response = requests.get(url, params=params, timeout=30, verify=False)
        
        if response.status_code == 200:
            data = response.json()
            if data and 'results' in data and data['results']:
                # Convertir a DataFrame
                df_tasas = pd.DataFrame(data['results'])
                df_tasas['fecha'] = pd.to_datetime(df_tasas['fecha'])
                df_tasas['Tasa_PlazoFijo_Diaria'] = (1 + df_tasas['valor']/100/365) - 1
                
                # Renombrar columna fecha
                df_tasas = df_tasas.rename(columns={'fecha': 'Fecha'})
                
                print(f"✅ Tasas históricas BCRA: {len(df_tasas)} registros")
                print(f"   Rango tasas: {df_tasas['valor'].min():.2f}% - {df_tasas['valor'].max():.2f}% TNA")
                
                return df_tasas[['Fecha', 'Tasa_PlazoFijo_Diaria']]
            else:
                print("❌ Sin datos en respuesta del BCRA")
        else:
            print(f"❌ Error HTTP BCRA: {response.status_code}")
    
    except Exception as e:
        print(f"⚠️  Error obteniendo tasas históricas BCRA: {str(e)}")
    
    return None

def cargar_todos_los_activos():
    """Carga datos del Excel Y SOLO agrega variables macroeconómicas reales"""
    
    print("CARGANDO DATASET COMPLETO...")
    
    # 1. Datos base del Excel (SIN MODIFICAR)
    ARCHIVO_DATOS = r"c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\Datos_historicos_de_la_cartera.xlsx"
    df_excel = pd.read_excel(ARCHIVO_DATOS, sheet_name='Hoja1')
    df_excel['Fecha'] = pd.to_datetime(df_excel['Fecha'])
    
    # Usar los activos del Excel PERO agregar los faltantes
    df_final = df_excel.copy()
    activos_excel = [col for col in df_final.columns 
                    if col != 'Fecha' and not col.startswith(('Retorno_', 'Tipo_', 'Tasa_'))]
    
    print(f"✅ Activos del Excel: {activos_excel}")
    
    # AGREGAR ACTIVOS FALTANTES que se perdieron
    activos_necesarios = ['BHIL', 'METRO', 'IBM']
    activos_faltantes = [a for a in activos_necesarios if a not in activos_excel]
    
    if activos_faltantes:
        print(f"🔍 Agregando activos faltantes: {activos_faltantes}")
        
        tickers_map = {
            'BHIL': 'BHIP.BA',  # Banco Hipotecario
            'METRO': 'METR.BA', # Banco Macro  
            'IBM': 'IBM'        # IBM directo
        }
        
        fecha_min = df_final['Fecha'].min()
        fecha_max = df_final['Fecha'].max()
        
        for activo in activos_faltantes:
            if activo in tickers_map:
                ticker = tickers_map[activo]
                try:
                    print(f"🔄 Descargando {activo} ({ticker})...")
                    data = yf.download(ticker, start=fecha_min, end=fecha_max + pd.Timedelta(days=1), progress=False)
                    
                    if len(data) > 0:
                        ticker_df = data['Close'].reset_index()
                        ticker_df.columns = ['Fecha', activo]
                        ticker_df['Fecha'] = pd.to_datetime(ticker_df['Fecha'])
                        
                        df_final = pd.merge(df_final, ticker_df, on='Fecha', how='outer')
                        print(f"✅ {activo}: {len(data)} registros agregados")
                    else:
                        print(f"❌ {activo}: Sin datos disponibles")
                        
                except Exception as e:
                    print(f"❌ Error con {activo}: {str(e)[:50]}")
    
    # Actualizar lista de activos
    activos_excel = [col for col in df_final.columns 
                    if col != 'Fecha' and not col.startswith(('Retorno_', 'Tipo_', 'Tasa_'))]
    
    # 2. AGREGAR SOLO VARIABLES MACROECONÓMICAS REALES
    print("\n💱 AGREGANDO DATOS MACROECONÓMICOS...")
    
    # Obtener período de datos del Excel
    fecha_min = df_final['Fecha'].min()
    fecha_max = df_final['Fecha'].max()
    
    # LIMPIAR VARIABLES MACRO EXISTENTES PRIMERO
    columnas_macro = ['Tipo_Cambio_ARSUSD', 'Retorno_FX', 'Tasa_PlazoFijo_Diaria']
    for col in columnas_macro:
        if col in df_final.columns:
            df_final = df_final.drop(columns=[col])
            print(f"🧹 Eliminada columna existente: {col}")
    
    # Obtener datos del dólar CCL - SOLO SI FUNCIONA  
    datos_ccl = obtener_dolar_ccl(fecha_min, fecha_max)
    if datos_ccl is not None:
        df_final = pd.merge(df_final, datos_ccl, on='Fecha', how='left')
        print(f"✅ Tipo_Cambio_ARSUSD y Retorno_FX agregados ({len(datos_ccl)} registros)")
    else:
        print("❌ CCL NO DISPONIBLE - Variables macro NO agregadas")
        # NO crear datos sintéticos - simplemente continuar sin estas variables
    
    # Agregar tasa de plazo fijo histórica SOLO si CCL funcionó
    if 'Tipo_Cambio_ARSUSD' in df_final.columns:
        datos_tasas = obtener_tasa_plazo_fijo_historica(fecha_min, fecha_max)
        if datos_tasas is not None:
            df_final = pd.merge(df_final, datos_tasas, on='Fecha', how='left')
            print(f"✅ Tasas históricas agregadas ({len(datos_tasas)} registros)")
        else:
            print("📊 Usando tasa fija estimada...")
            tasa_diaria = (1 + 1.07) ** (1/365) - 1  # 107% anual
            df_final['Tasa_PlazoFijo_Diaria'] = tasa_diaria
    else:
        print("❌ Tasa PF NO agregada (CCL no disponible)")
    
    # 3. Finalizar dataset
    df_final = df_final.sort_values('Fecha').reset_index(drop=True)
    activos_finales = [col for col in df_final.columns if col != 'Fecha']
    
    print(f"\n🎯 DATASET FINAL:")
    print(f"   • Activos originales: {len(activos_excel)}")
    variables_macro = [col for col in activos_finales if col.startswith(('Retorno_', 'Tipo_', 'Tasa_'))]
    print(f"   • Variables macro agregadas: {len(variables_macro)} {variables_macro}")
    print(f"   • Total columnas: {len(activos_finales)}")
    print(f"   • Registros: {len(df_final)}")
    print(f"   • Período: {df_final['Fecha'].min().strftime('%d/%m/%Y')} → {df_final['Fecha'].max().strftime('%d/%m/%Y')}")
    
    return df_final, activos_finales

# Ejecutar carga completa
df_data_completo, activos_completos = cargar_todos_los_activos()

# Verificar integridad de datos
print(f"\n📊 VERIFICACIÓN DE DATOS:")
for activo in activos_completos:
    valid_count = df_data_completo[activo].notna().sum()
    tipo_variable = "📈 Activo" if not activo.startswith(('Retorno_', 'Tipo_', 'Tasa_')) else "💱 Macro"
    print(f"   {tipo_variable} {activo:20s}: {valid_count:3d} registros válidos")

# ACTUALIZAR ARCHIVO EXCEL SIN ROMPER FECHAS
try:
    variables_macro = [col for col in df_data_completo.columns if col.startswith(('Retorno_', 'Tipo_', 'Tasa_'))]
    
    if variables_macro:
        archivo_datos = r"c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\Datos_historicos_de_la_cartera.xlsx"
        
        # PRESERVAR FORMATO DE FECHA
        df_guardar = df_data_completo.copy()
        df_guardar['Fecha'] = df_guardar['Fecha'].dt.strftime('%Y-%m-%d')
        
        with pd.ExcelWriter(archivo_datos, engine='openpyxl') as writer:
            df_guardar.to_excel(writer, sheet_name='Hoja1', index=False)
        
        print(f"\n💾 ACTUALIZADO: Excel con variables macro: {variables_macro}")
        print(f"   ✅ Formato de fecha preservado")
    else:
        print(f"\n❌ Excel NO actualizado (no hay variables macro para agregar)")
        
except Exception as e:
    print(f"❌ Error actualizando archivo: {str(e)}")

CARGANDO DATASET COMPLETO...
✅ Activos del Excel: ['AAPL', 'SPY', 'EWZ', 'CEPU']
🔍 Agregando activos faltantes: ['BHIL', 'METRO', 'IBM']
🔄 Descargando BHIL (BHIP.BA)...
✅ BHIL: 161 registros agregados
🔄 Descargando METRO (METR.BA)...
✅ METRO: 161 registros agregados
🔄 Descargando IBM (IBM)...
✅ IBM: 165 registros agregados

💱 AGREGANDO DATOS MACROECONÓMICOS...
🧹 Eliminada columna existente: Tipo_Cambio_ARSUSD
🧹 Eliminada columna existente: Retorno_FX
🧹 Eliminada columna existente: Tasa_PlazoFijo_Diaria
🔄 Obteniendo datos históricos del Dólar CCL (ArgentinaDatos)...
✅ METRO: 161 registros agregados
🔄 Descargando IBM (IBM)...
✅ IBM: 165 registros agregados

💱 AGREGANDO DATOS MACROECONÓMICOS...
🧹 Eliminada columna existente: Tipo_Cambio_ARSUSD
🧹 Eliminada columna existente: Retorno_FX
🧹 Eliminada columna existente: Tasa_PlazoFijo_Diaria
🔄 Obteniendo datos históricos del Dólar CCL (ArgentinaDatos)...
✅ CCL histórico obtenido: 243 registros
   Período: 2025-01-03 → 2025-09-02
   Rango CCL: 

## 2. Análisis de Riesgo y Performance Avanzado

Cálculo completo de métricas financieras: Sharpe, Sortino, Alpha, Beta, Treynor, Kelly, VaR, CVaR y más.

In [174]:
class RiskCalculator:
    """Calculador profesional de métricas de riesgo y performance avanzadas"""
    
    def __init__(self, risk_free_rate=0.02, benchmark_symbol='SPY'):
        self.rf_rate = risk_free_rate / 252  # Convertir a tasa diaria
        self.annual_rf = risk_free_rate
        self.benchmark = benchmark_symbol
        self.use_dynamic_rf = False  # Flag para usar tasa dinámica
    
    def set_dynamic_risk_free_rate(self, df):
        """Configura tasa libre de riesgo dinámica desde datos"""
        if 'Tasa_PlazoFijo_Diaria' in df.columns:
            self.dynamic_rf = df['Tasa_PlazoFijo_Diaria'].mean()  # Promedio de tasas históricas
            self.annual_rf = self.dynamic_rf * 365
            self.rf_rate = self.dynamic_rf
            self.use_dynamic_rf = True
            print(f"✅ Usando tasa libre de riesgo REAL del BCRA: {self.annual_rf:.2%} anual")
        else:
            print("⚠️  Tasa_PlazoFijo_Diaria no disponible, usando tasa fija")
    
    def calculate_fx_metrics(self, returns, df):
        """Calcula métricas de exposición al riesgo cambiario"""
        fx_metrics = {}
        
        if 'Retorno_FX' in df.columns and 'Tipo_Cambio_ARSUSD' in df.columns:
            fx_returns = df['Retorno_FX'].dropna()
            
            # Alinear series de tiempo
            common_dates = returns.index.intersection(fx_returns.index)
            if len(common_dates) > 30:
                aligned_returns = returns.loc[common_dates]
                aligned_fx = fx_returns.loc[common_dates]
                
                # Correlación con tipo de cambio
                correlation = np.corrcoef(aligned_returns, aligned_fx)[0, 1]
                fx_metrics['fx_correlation'] = correlation if not np.isnan(correlation) else 0
                
                # Beta FX (sensibilidad al tipo de cambio)
                covariance = np.cov(aligned_returns, aligned_fx)[0, 1]
                fx_variance = np.var(aligned_fx)
                fx_metrics['fx_beta'] = covariance / fx_variance if fx_variance > 0 else 0
                
                # Volatilidad durante crisis cambiarias (retorno FX > 2 std)
                crisis_threshold = aligned_fx.mean() + 2 * aligned_fx.std()
                crisis_periods = aligned_fx > crisis_threshold
                if crisis_periods.any():
                    crisis_vol = aligned_returns[crisis_periods].std() * np.sqrt(252)
                    normal_vol = aligned_returns[~crisis_periods].std() * np.sqrt(252)
                    fx_metrics['crisis_volatility_ratio'] = crisis_vol / normal_vol if normal_vol > 0 else 1
                else:
                    fx_metrics['crisis_volatility_ratio'] = 1
            else:
                fx_metrics = {'fx_correlation': 0, 'fx_beta': 0, 'crisis_volatility_ratio': 1}
        else:
            fx_metrics = {'fx_correlation': None, 'fx_beta': None, 'crisis_volatility_ratio': None}
        
        return fx_metrics
    
    def get_benchmark_returns(self, df, asset_dates=None):
        """Obtiene retornos del benchmark"""
        if self.benchmark not in df.columns:
            # Crear benchmark sintético si no existe
            available_assets = [col for col in df.columns if col != 'Fecha']
            if len(available_assets) >= 3:
                market_prices = df[available_assets[:3]].mean(axis=1, skipna=True)
                market_returns = market_prices.pct_change().dropna()
                return market_returns.values
        else:
            benchmark_prices = df[self.benchmark].dropna()
            benchmark_returns = benchmark_prices.pct_change().dropna()
            return benchmark_returns.values
        return None
    
    def calculate_alpha_beta(self, asset_returns, benchmark_returns):
        """Calcula Alpha y Beta usando numpy (evitamos sklearn)"""
        if len(asset_returns) != len(benchmark_returns):
            min_len = min(len(asset_returns), len(benchmark_returns))
            asset_returns = asset_returns[:min_len]
            benchmark_returns = benchmark_returns[:min_len]
        
        # Filtrar NaN
        mask = ~(np.isnan(asset_returns) | np.isnan(benchmark_returns))
        asset_returns = asset_returns[mask]
        benchmark_returns = benchmark_returns[mask]
        
        if len(asset_returns) < 30:
            return None, None, None
        
        try:
            # Regresión lineal simple usando numpy
            # Beta = Cov(R_asset, R_market) / Var(R_market)
            covariance = np.cov(asset_returns, benchmark_returns)[0, 1]
            market_variance = np.var(benchmark_returns)
            
            if market_variance == 0:
                return None, None, None
            
            beta = covariance / market_variance
            
            # Alpha = R_asset_mean - Beta * R_market_mean
            alpha_daily = np.mean(asset_returns) - beta * np.mean(benchmark_returns)
            alpha_annual = alpha_daily * 252
            
            # R-squared
            correlation = np.corrcoef(asset_returns, benchmark_returns)[0, 1]
            r_squared = correlation ** 2 if not np.isnan(correlation) else 0
            
            return alpha_annual, beta, r_squared
            
        except Exception:
            return None, None, None
    
    def calculate_advanced_metrics(self, returns, df=None, asset_name=None):
        """Calcula métricas avanzadas adicionales"""
        metrics = {}
        
        # Inicializar con None para asegurar consistencia
        metrics['alpha'] = None
        metrics['beta'] = None
        metrics['r_squared'] = None
        metrics['treynor_ratio'] = None
        metrics['information_ratio'] = None
        metrics['tracking_error'] = None
        
        # Treynor Ratio y métricas vs benchmark
        if df is not None:
            benchmark_returns = self.get_benchmark_returns(df)
            if benchmark_returns is not None and len(benchmark_returns) > 30:
                try:
                    alpha, beta, r_squared = self.calculate_alpha_beta(returns.values, benchmark_returns)
                    
                    if alpha is not None and beta is not None and r_squared is not None:
                        metrics['alpha'] = alpha
                        metrics['beta'] = beta
                        metrics['r_squared'] = r_squared
                        
                        # Treynor Ratio
                        if beta is not None and abs(beta) > 0.001:  # Evitar división por cero
                            excess_return = returns.mean() * 252 - self.annual_rf
                            metrics['treynor_ratio'] = excess_return / beta
                        
                        # Information Ratio y Tracking Error
                        min_len = min(len(returns), len(benchmark_returns))
                        if min_len > 30:
                            asset_trim = returns.values[:min_len]
                            bench_trim = benchmark_returns[:min_len]
                            
                            # Filtrar NaNs
                            mask = ~(np.isnan(asset_trim) | np.isnan(bench_trim))
                            if mask.sum() > 30:
                                excess_returns = asset_trim[mask] - bench_trim[mask]
                                tracking_error = np.std(excess_returns) * np.sqrt(252)
                                metrics['tracking_error'] = tracking_error
                                
                                if tracking_error > 0:
                                    metrics['information_ratio'] = (np.mean(excess_returns) * 252) / tracking_error
                                    
                except Exception as e:
                    print(f"⚠️  Error calculando métricas benchmark para {asset_name}: {str(e)[:50]}")
        
        # Kelly Criterion
        positive_returns = returns[returns > 0]
        negative_returns = returns[returns < 0]
        
        if len(positive_returns) > 0 and len(negative_returns) > 0:
            avg_win = positive_returns.mean()
            avg_loss = abs(negative_returns.mean())
            win_rate = len(positive_returns) / len(returns)
            
            if avg_loss > 0:
                b = avg_win / avg_loss
                kelly_f = (b * win_rate - (1 - win_rate)) / b
                metrics['kelly_criterion'] = max(0, min(kelly_f, 1))
            else:
                metrics['kelly_criterion'] = None
        else:
            metrics['kelly_criterion'] = None
        
        # Calmar Ratio
        cumulative = (1 + returns).cumprod()
        running_max = cumulative.cummax()
        drawdown = (cumulative - running_max) / running_max
        max_dd = drawdown.min()
        
        if max_dd < 0:
            annual_return = returns.mean() * 252
            metrics['calmar_ratio'] = annual_return / abs(max_dd)
        else:
            metrics['calmar_ratio'] = None
        
        # Ulcer Index
        drawdown_squared = (drawdown * 100) ** 2
        metrics['ulcer_index'] = np.sqrt(drawdown_squared.mean())
        
        return metrics
    
    def calculate_metrics(self, returns, df=None, asset_name=None):
        """Calcula métricas completas de riesgo y rendimiento"""
        if len(returns) < 30:
            return None
        
        # Limpiar datos
        returns = returns.dropna()
        returns = returns.replace([np.inf, -np.inf], np.nan).dropna()
        
        if len(returns) < 30:
            return None
        
        # Métricas básicas
        annual_return = returns.mean() * 252
        volatility = returns.std() * np.sqrt(252)
        
        # Sharpe Ratio
        excess_return = annual_return - self.annual_rf
        sharpe = excess_return / volatility if volatility > 0 else 0
        
        # Sortino Ratio
        negative_returns = returns[returns < 0]
        downside_deviation = negative_returns.std() * np.sqrt(252) if len(negative_returns) > 0 else volatility
        sortino = excess_return / downside_deviation if downside_deviation > 0 else 0
        
        # VaR y CVaR
        var_95 = np.percentile(returns, 5)
        cvar_95 = returns[returns <= var_95].mean()
        
        # Maximum Drawdown
        cumulative = (1 + returns).cumprod()
        running_max = cumulative.cummax()
        drawdown = (cumulative - running_max) / running_max
        max_dd = drawdown.min()
        
        # Métricas básicas
        basic_metrics = {
            'rendimiento_anual': annual_return,
            'volatilidad': volatility,
            'sharpe_ratio': sharpe,
            'sortino_ratio': sortino,
            'var_95': var_95,
            'cvar_95': cvar_95,
            'max_drawdown': max_dd,
            'skewness': returns.skew(),
            'kurtosis': returns.kurtosis()
        }
        
        # Métricas avanzadas
        advanced_metrics = self.calculate_advanced_metrics(returns, df, asset_name)
        
        # Combinar ambos diccionarios
        basic_metrics.update(advanced_metrics)
        
        # AGREGAR MÉTRICAS FX si hay datos macro disponibles
        if df is not None:
            fx_metrics = self.calculate_fx_metrics(returns, df)
            basic_metrics.update(fx_metrics)
        
        return basic_metrics

def analizar_activos(df, activos):
    """Analiza todos los activos y retorna métricas completas"""
    calculator = RiskCalculator()
    
    # CONFIGURAR TASA LIBRE DE RIESGO DINÁMICA
    calculator.set_dynamic_risk_free_rate(df)
    
    resultados = []
    
    for i, activo in enumerate(activos):
        precios = df[activo].dropna()
        if len(precios) > 30:
            returns = precios.pct_change().dropna()
            metricas = calculator.calculate_metrics(returns, df, activo)
            if metricas:
                metricas['activo'] = activo
                metricas['observaciones'] = len(returns)
                resultados.append(metricas)
                
                # Mostrar progreso incluyendo métricas FX
                if i < 5:
                    alpha_str = f"α={metricas['alpha']:.4f}" if metricas.get('alpha') else "α=N/A"
                    beta_str = f"β={metricas['beta']:.3f}" if metricas.get('beta') else "β=N/A"
                    fx_corr_str = f"FX_ρ={metricas.get('fx_correlation', 0):.3f}" if metricas.get('fx_correlation') is not None else "FX_ρ=N/A"
                    print(f"   ✅ {activo}: Sharpe={metricas['sharpe_ratio']:.3f}, {alpha_str}, {beta_str}, {fx_corr_str}")
    
    if not resultados:
        return pd.DataFrame()
    
    df_metricas = pd.DataFrame(resultados)
    
    # Reordenar columnas por importancia
    basic_cols = ['activo', 'observaciones', 'rendimiento_anual', 'volatilidad', 
                 'sharpe_ratio', 'sortino_ratio', 'max_drawdown']
    advanced_cols = ['alpha', 'beta', 'r_squared', 'treynor_ratio', 'information_ratio', 
                    'tracking_error', 'kelly_criterion', 'calmar_ratio', 'ulcer_index']
    risk_cols = ['var_95', 'cvar_95', 'skewness', 'kurtosis']
    
    # Verificar qué columnas existen
    available_cols = []
    for col_group in [basic_cols, advanced_cols, risk_cols]:
        available_cols.extend([col for col in col_group if col in df_metricas.columns])
    
    # Agregar columnas restantes
    remaining_cols = [col for col in df_metricas.columns if col not in available_cols]
    all_cols = available_cols + remaining_cols
    
    df_metricas = df_metricas[all_cols]
    
    return df_metricas

# Calcular métricas usando el dataset completo con variables macro
# Filtrar solo activos (excluir variables macro)
activos_reales = [col for col in activos_completos if not col.startswith(('Retorno_', 'Tipo_', 'Tasa_'))]
print("🔬 Iniciando análisis completo de métricas de riesgo...")
df_metricas = analizar_activos(df_data_completo, activos_reales)

# Mostrar resultados organizados
if not df_metricas.empty:
    df_display = df_metricas.copy()
    
    # Formatear porcentajes
    percentage_cols = ['rendimiento_anual', 'volatilidad', 'max_drawdown', 'var_95', 'cvar_95', 'alpha', 'tracking_error']
    for col in percentage_cols:
        if col in df_display.columns:
            df_display[col] = (df_display[col] * 100).round(2)
    
    # Formatear ratios
    ratio_cols = ['sharpe_ratio', 'sortino_ratio', 'treynor_ratio', 'information_ratio', 
                  'calmar_ratio', 'kelly_criterion', 'beta', 'r_squared', 'ulcer_index']
    for col in ratio_cols:
        if col in df_display.columns:
            df_display[col] = df_display[col].round(3)
    
    print(f"\n📋 REPORTE COMPLETO DE MÉTRICAS (n={len(df_metricas)} activos)")
    print("="*90)
    
    # Tabla 1: Métricas principales
    main_cols = ['activo', 'rendimiento_anual', 'volatilidad', 'sharpe_ratio', 'max_drawdown']
    available_main = [col for col in main_cols if col in df_display.columns]
    if available_main:
        print("\n🔸 PERFORMANCE Y RIESGO BÁSICO:")
        display(df_display[available_main])
    
    # Tabla 2: Métricas vs Benchmark (Alpha, Beta, etc.)
    benchmark_cols = ['activo', 'alpha', 'beta', 'r_squared', 'treynor_ratio', 'information_ratio', 'tracking_error']
    available_benchmark = [col for col in benchmark_cols if col in df_display.columns]
    
    # Solo mostrar si hay al menos algunos valores no nulos
    has_benchmark_data = any(df_display[col].notna().any() for col in available_benchmark[1:] if col in df_display.columns)
    if available_benchmark and has_benchmark_data:
        print("\n🔸 MÉTRICAS vs BENCHMARK:")
        display(df_display[available_benchmark])
    
    # Tabla 3: Métricas FX (nuevas)
    fx_cols = ['activo', 'fx_correlation', 'fx_beta', 'crisis_volatility_ratio']
    available_fx = [col for col in fx_cols if col in df_display.columns]
    has_fx_data = any(df_display[col].notna().any() for col in available_fx[1:] if col in df_display.columns)
    if available_fx and has_fx_data:
        print("\n💱 MÉTRICAS DE RIESGO CAMBIARIO (CCL):")
        # Formatear las nuevas métricas FX
        fx_display = df_display[available_fx].copy()
        for col in ['fx_correlation', 'fx_beta']:
            if col in fx_display.columns:
                fx_display[col] = fx_display[col].round(3)
        if 'crisis_volatility_ratio' in fx_display.columns:
            fx_display['crisis_volatility_ratio'] = fx_display['crisis_volatility_ratio'].round(2)
        display(fx_display)
    
    # Tabla 4: Métricas avanzadas y especializadas
    advanced_cols = ['activo', 'sortino_ratio', 'kelly_criterion', 'calmar_ratio', 'ulcer_index']
    available_advanced = [col for col in advanced_cols if col in df_display.columns]
    if available_advanced:
        print("\n🔸 MÉTRICAS AVANZADAS:")
        display(df_display[available_advanced])
    
    # Análisis de Top Performers
    print("\n🏆 TOP PERFORMERS:")
    print("─"*60)
    
    # Mejor Sharpe Ratio
    if 'sharpe_ratio' in df_metricas.columns:
        mejor_sharpe = df_metricas.loc[df_metricas['sharpe_ratio'].idxmax()]
        print(f"📈 Mejor Sharpe Ratio: {mejor_sharpe['activo']} ({mejor_sharpe['sharpe_ratio']:.3f})")
    
    # Mejor Alpha (si disponible)
    alpha_data = df_metricas.dropna(subset=['alpha']) if 'alpha' in df_metricas.columns else pd.DataFrame()
    if not alpha_data.empty:
        mejor_alpha = alpha_data.loc[alpha_data['alpha'].idxmax()]
        print(f"🎯 Mejor Alpha: {mejor_alpha['activo']} ({mejor_alpha['alpha']*100:.2f}%)")
    
    # Mejor Kelly Criterion
    kelly_data = df_metricas.dropna(subset=['kelly_criterion']) if 'kelly_criterion' in df_metricas.columns else pd.DataFrame()
    if not kelly_data.empty:
        mejor_kelly = kelly_data.loc[kelly_data['kelly_criterion'].idxmax()]
        print(f"🎲 Mejor Kelly: {mejor_kelly['activo']} ({mejor_kelly['kelly_criterion']:.3f})")
    
    # Menor riesgo (máximo drawdown más bajo)
    menor_riesgo = df_metricas.loc[df_metricas['max_drawdown'].idxmax()]  # idxmax porque son valores negativos
    print(f"🛡️  Menor Riesgo: {menor_riesgo['activo']} (DD: {menor_riesgo['max_drawdown']*100:.2f}%)")
    
    # ACTUALIZAR EXCEL EXISTENTE CON DATOS CALCULADOS
    try:
        archivo_ratios = r"c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\Ratios Managment financiero cartera.xlsx"
        
        # Leer archivo existente para mantener hoja Drawdown
        with pd.ExcelFile(archivo_ratios) as existing_file:
            existing_data = {}
            for sheet in existing_file.sheet_names:
                existing_data[sheet] = pd.read_excel(existing_file, sheet_name=sheet)
        
        # Actualizar archivo con datos calculados
        with pd.ExcelWriter(archivo_ratios, engine='openpyxl') as writer:
            # ACTUALIZAR Drawdown con datos calculados
            drawdown_data = df_metricas[['activo', 'max_drawdown', 'ulcer_index', 'calmar_ratio']].copy()
            drawdown_data['max_drawdown'] = drawdown_data['max_drawdown'] * 100  # Convertir a porcentaje
            drawdown_data.columns = ['Activo', 'Max_Drawdown_%', 'Ulcer_Index', 'Calmar_Ratio']
            drawdown_data.to_excel(writer, sheet_name='Drawdown', index=False)
            
            # ACTUALIZAR hojas específicas con datos calculados
            # VaR - Actualizar con datos de VaR calculados
            var_data = df_metricas[['activo', 'var_95', 'cvar_95']].copy()
            var_data['var_95'] = var_data['var_95'] * 100  # Convertir a porcentaje
            var_data['cvar_95'] = var_data['cvar_95'] * 100
            var_data.columns = ['Activo', 'VaR_95%', 'CVaR_95%']
            var_data.to_excel(writer, sheet_name='VaR', index=False)
            
            # VaR cond - Usar CVaR
            cvar_data = df_metricas[['activo', 'cvar_95']].copy()
            cvar_data['cvar_95'] = cvar_data['cvar_95'] * 100
            cvar_data.columns = ['Activo', 'CVaR_95%']
            cvar_data.to_excel(writer, sheet_name='VaR cond', index=False)
            
            # Kelly criterion
            kelly_data_sheet = df_metricas[['activo', 'kelly_criterion']].copy()
            kelly_data_sheet.columns = ['Activo', 'Kelly_Criterion']
            kelly_data_sheet.to_excel(writer, sheet_name='Kelly criterion', index=False)
            
            # Sharpe
            sharpe_data = df_metricas[['activo', 'sharpe_ratio', 'rendimiento_anual', 'volatilidad']].copy()
            sharpe_data['rendimiento_anual'] = sharpe_data['rendimiento_anual'] * 100
            sharpe_data['volatilidad'] = sharpe_data['volatilidad'] * 100
            sharpe_data.columns = ['Activo', 'Sharpe_Ratio', 'Rendimiento_Anual_%', 'Volatilidad_%']
            sharpe_data.to_excel(writer, sheet_name='Sharpe', index=False)
            
            # Sortino
            sortino_data = df_metricas[['activo', 'sortino_ratio', 'rendimiento_anual', 'max_drawdown']].copy()
            sortino_data['rendimiento_anual'] = sortino_data['rendimiento_anual'] * 100
            sortino_data['max_drawdown'] = sortino_data['max_drawdown'] * 100
            sortino_data.columns = ['Activo', 'Sortino_Ratio', 'Rendimiento_Anual_%', 'Max_Drawdown_%']
            sortino_data.to_excel(writer, sheet_name='Sortino', index=False)
            
            # Alpha beta
            alpha_beta_data = df_metricas[['activo', 'alpha', 'beta', 'r_squared', 'treynor_ratio']].copy()
            alpha_beta_data['alpha'] = alpha_beta_data['alpha'] * 100
            alpha_beta_data.columns = ['Activo', 'Alpha_%', 'Beta', 'R_Squared', 'Treynor_Ratio']
            alpha_beta_data.to_excel(writer, sheet_name='Alpha beta', index=False)
        
        print(f"\n💾 ACTUALIZADO: '{archivo_ratios}' - TODAS LAS HOJAS ACTUALIZADAS con datos calculados")
        
    except Exception as e:
        print(f"❌ Error actualizando archivo de ratios: {str(e)}")
    
else:
    print("❌ No se pudieron calcular métricas")

🔬 Iniciando análisis completo de métricas de riesgo...
✅ Usando tasa libre de riesgo REAL del BCRA: 32.87% anual
   ✅ AAPL: Sharpe=-0.282, α=-0.2122, β=1.087, FX_ρ=0.285
   ✅ SPY: Sharpe=0.241, α=-0.0025, β=1.006, FX_ρ=0.242
   ✅ EWZ: Sharpe=1.202, α=0.3945, β=0.724, FX_ρ=0.233
   ✅ CEPU: Sharpe=-1.251, α=-0.5598, β=0.584, FX_ρ=0.069
   ✅ BHIL: Sharpe=-2.084, α=-1.0973, β=0.467, FX_ρ=-0.059

📋 REPORTE COMPLETO DE MÉTRICAS (n=7 activos)

🔸 PERFORMANCE Y RIESGO BÁSICO:


,activo,rendimiento_anual,volatilidad,sharpe_ratio,max_drawdown
0,AAPL,21.8000,39.2600,-0.2820,-27.6900
1,SPY,39.5600,27.7200,0.2410,-22.0400
2,EWZ,68.1100,29.3100,1.2020,-17.8900
3,CEPU,-32.8500,52.5500,-1.2510,-35.4200
4,BHIL,-91.2600,59.5800,-2.0840,-51.4200
5,METRO,-71.4600,74.0800,-1.4080,-50.6800
6,IBM,20.5900,31.9900,-0.3840,-19.8200



🔸 MÉTRICAS vs BENCHMARK:


,activo,alpha,beta,r_squared,treynor_ratio,information_ratio,tracking_error
0,AAPL,-21.2200,1.0870,0.5820,-0.1020,-0.7000,25.4000
1,SPY,-0.2500,1.0060,1.0000,0.0670,NaN,0.0000
2,EWZ,39.4500,0.7240,0.4630,0.4870,1.2540,22.7700
3,CEPU,-55.9800,0.5840,0.0940,-1.1250,-1.4140,51.2000
4,BHIL,-109.7300,0.4670,0.0470,-2.6590,-2.1860,59.8500
5,METRO,-90.6500,0.4850,0.0330,-2.1510,-1.5000,74.0300
6,IBM,23.0700,-0.0320,0.0010,3.8930,-0.4130,43.0100



💱 MÉTRICAS DE RIESGO CAMBIARIO (CCL):


,activo,fx_correlation,fx_beta,crisis_volatility_ratio
0,AAPL,0.2850,0.3800,NaN
1,SPY,0.2420,0.2280,NaN
2,EWZ,0.2330,0.2320,NaN
3,CEPU,0.0690,0.1230,NaN
4,BHIL,-0.0590,-0.1190,NaN
5,METRO,-0.1260,-0.3160,NaN
6,IBM,-0.0550,-0.0460,NaN



🔸 MÉTRICAS AVANZADAS:


,activo,sortino_ratio,kelly_criterion,calmar_ratio,ulcer_index
0,AAPL,-0.3860,0.0000,0.7870,13.1220
1,SPY,0.3100,0.0960,1.7950,5.6010
2,EWZ,1.8640,0.1280,3.8060,4.8580
3,CEPU,-2.3170,0.0000,-0.9280,22.1010
4,BHIL,-3.7130,0.0000,-1.7750,31.9780
5,METRO,-2.7630,0.0000,-1.4100,28.5010
6,IBM,-0.5550,0.0620,1.0390,8.2850



🏆 TOP PERFORMERS:
────────────────────────────────────────────────────────────
📈 Mejor Sharpe Ratio: EWZ (1.202)
🎯 Mejor Alpha: EWZ (39.45%)
🎲 Mejor Kelly: EWZ (0.128)
🛡️  Menor Riesgo: EWZ (DD: -17.89%)

💾 ACTUALIZADO: 'c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\Ratios Managment financiero cartera.xlsx' - TODAS LAS HOJAS ACTUALIZADAS con datos calculados

💾 ACTUALIZADO: 'c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\Ratios Managment financiero cartera.xlsx' - TODAS LAS HOJAS ACTUALIZADAS con datos calculados


## 3. Visualizaciones y Análisis de Correlación

Gráficos profesionales para análisis de performance y correlaciones.

In [175]:
def crear_visualizaciones(df, activos):
    """Crear visualizaciones profesionales"""
    
    # 1. Performance normalizada
    fig_perf = go.Figure()
    colors = px.colors.qualitative.Set3
    
    for i, activo in enumerate(activos):
        precios = df[activo].dropna()
        if len(precios) > 10:
            fechas = df.loc[df[activo].notna(), 'Fecha']
            perf_norm = (precios / precios.iloc[0]) * 100
            
            fig_perf.add_trace(go.Scatter(
                x=fechas, y=perf_norm, name=activo,
                line=dict(width=2, color=colors[i % len(colors)]),
                hovertemplate=f'{activo}: %{{y:.1f}}<extra></extra>'
            ))
    
    fig_perf.update_layout(
        title='Performance Normalizada de Activos (Base 100)',
        xaxis_title='Fecha', yaxis_title='Performance',
        template='plotly_white', height=500, hovermode='x unified'
    )
    # Mostrar y guardar como imagen estática
    fig_perf.show()
    fig_perf.write_image("performance_normalizada.png", width=1000, height=500, scale=2)
    
    # 2. Mapa Riesgo vs Rendimiento
    if not df_metricas.empty:
        fig_risk = go.Figure()
        
        for i, row in df_metricas.iterrows():
            fig_risk.add_trace(go.Scatter(
                x=[row['volatilidad'] * 100],
                y=[row['rendimiento_anual'] * 100],
                mode='markers+text',
                text=[row['activo']],
                textposition="top center",
                name=row['activo'],
                marker=dict(size=12, color=colors[i % len(colors)]),
                showlegend=False,
                hovertemplate=f"<b>{row['activo']}</b><br>Rendimiento: {row['rendimiento_anual']*100:.1f}%<br>Volatilidad: {row['volatilidad']*100:.1f}%<extra></extra>"
            ))
        
        fig_risk.update_layout(
            title='Mapa Riesgo vs Rendimiento',
            xaxis_title='Volatilidad Anual (%)',
            yaxis_title='Rendimiento Anual (%)',
            template='plotly_white', height=500
        )
        fig_risk.add_hline(y=0, line_dash="dash", line_color="gray")
        # Mostrar y guardar como imagen estática
        fig_risk.show()
        fig_risk.write_image("mapa_riesgo_rendimiento.png", width=1000, height=500, scale=2)
    
    # 3. Matriz de correlación
    returns_matrix = pd.DataFrame()
    for activo in activos:
        precios = df[activo].dropna()
        if len(precios) > 30:
            returns = precios.pct_change().dropna()
            returns_matrix[activo] = returns
    
    if len(returns_matrix.columns) > 1:
        corr_matrix = returns_matrix.corr()
        
        fig_corr = go.Figure(data=go.Heatmap(
            z=corr_matrix.values,
            x=corr_matrix.columns,
            y=corr_matrix.index,
            colorscale='RdBu', zmid=0,
            text=corr_matrix.round(2).values,
            texttemplate='%{text}',
            hovertemplate='%{y} vs %{x}: %{z:.3f}<extra></extra>'
        ))
        
        fig_corr.update_layout(
            title='Matriz de Correlación de Retornos',
            template='plotly_white', height=400
        )
        # Mostrar y guardar como imagen estática
        fig_corr.show()
        fig_corr.write_image("matriz_correlacion.png", width=800, height=400, scale=2)
        
        # Estadísticas de correlación
        corr_mean = corr_matrix.mean().mean()
        print(f"📊 Correlación promedio: {corr_mean:.3f}")
        
        return returns_matrix
    
    return pd.DataFrame()

# Crear visualizaciones
print("📊 Generando visualizaciones...")
returns_data = crear_visualizaciones(df_data, activos)

📊 Generando visualizaciones...


📊 Correlación promedio: 0.469


## 4. Optimización de Cartera

Cálculo de carteras óptimas usando teoría moderna de portafolios.

In [176]:
class PortfolioOptimizer:
    """Optimización de carteras usando Markowitz"""
    
    def __init__(self, returns_data, risk_free_rate=0.02):
        self.returns = returns_data.dropna()
        self.assets = self.returns.columns.tolist()
        self.n_assets = len(self.assets)
        self.rf_rate = risk_free_rate / 252
        
    def negative_sharpe(self, weights):
        """Función objetivo para maximizar Sharpe"""
        portfolio_return = np.sum(self.returns.mean() * weights) * 252
        portfolio_std = np.sqrt(np.dot(weights.T, np.dot(self.returns.cov() * 252, weights)))
        sharpe = (portfolio_return - self.rf_rate * 252) / portfolio_std
        return -sharpe
    
    def optimize_sharpe(self):
        """Optimiza para máximo Sharpe Ratio"""
        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        bounds = tuple((0, 1) for _ in range(self.n_assets))
        initial = np.array([1/self.n_assets] * self.n_assets)
        
        try:
            result = minimize(self.negative_sharpe, initial, method='SLSQP', 
                            bounds=bounds, constraints=constraints)
            
            if result.success:
                weights = dict(zip(self.assets, result.x))
                portfolio_return = np.sum(self.returns.mean() * result.x) * 252
                portfolio_std = np.sqrt(np.dot(result.x.T, np.dot(self.returns.cov() * 252, result.x)))
                sharpe = (portfolio_return - self.rf_rate * 252) / portfolio_std
                
                return {
                    'tipo': 'Máximo Sharpe',
                    'weights': weights,
                    'rendimiento': portfolio_return,
                    'volatilidad': portfolio_std,
                    'sharpe_ratio': sharpe
                }
        except:
            pass
        return None
    
    def optimize_min_vol(self):
        """Optimiza para mínima volatilidad"""
        def portfolio_vol(weights):
            return np.sqrt(np.dot(weights.T, np.dot(self.returns.cov() * 252, weights)))
        
        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        bounds = tuple((0, 1) for _ in range(self.n_assets))
        initial = np.array([1/self.n_assets] * self.n_assets)
        
        try:
            result = minimize(portfolio_vol, initial, method='SLSQP',
                            bounds=bounds, constraints=constraints)
            
            if result.success:
                weights = dict(zip(self.assets, result.x))
                portfolio_return = np.sum(self.returns.mean() * result.x) * 252
                portfolio_std = result.fun
                sharpe = (portfolio_return - self.rf_rate * 252) / portfolio_std
                
                return {
                    'tipo': 'Mínimo Riesgo',
                    'weights': weights,
                    'rendimiento': portfolio_return,
                    'volatilidad': portfolio_std,
                    'sharpe_ratio': sharpe
                }
        except:
            pass
        return None

def optimizar_carteras(returns_data):
    """Ejecuta optimización y muestra resultados"""
    if len(returns_data.columns) < 2:
        print("❌ Necesario mínimo 2 activos para optimización")
        return None, None
    
    # OBTENER LA TASA LIBRE DE RIESGO REAL DEL BCRA
    tasa_real_bcra = 0.02  # Default fallback
    if 'df_data_completo' in globals() and 'Tasa_PlazoFijo_Diaria' in df_data_completo.columns:
        # Convertir tasa diaria BCRA a anual para el optimizador
        tasa_diaria_promedio = df_data_completo['Tasa_PlazoFijo_Diaria'].dropna().mean()
        tasa_real_bcra = ((1 + tasa_diaria_promedio) ** 252) - 1
        print(f"✅ Usando tasa libre de riesgo REAL del BCRA: {tasa_real_bcra*100:.2f}% anual")
    else:
        print(f"⚠️ Usando tasa fija del 2% (datos BCRA no disponibles)")
    
    optimizer = PortfolioOptimizer(returns_data, risk_free_rate=tasa_real_bcra)
    
    # Optimizar carteras
    cartera_sharpe = optimizer.optimize_sharpe()
    cartera_min_vol = optimizer.optimize_min_vol()
    
    carteras = []
    if cartera_sharpe: carteras.append(cartera_sharpe)
    if cartera_min_vol: carteras.append(cartera_min_vol)
    
    # Mostrar resultados
    print("🎯 Carteras Optimizadas:")
    print("─" * 60)
    
    for cartera in carteras:
        print(f"\n📊 {cartera['tipo']}:")
        print(f"   Rendimiento: {cartera['rendimiento']*100:5.2f}%")
        print(f"   Volatilidad: {cartera['volatilidad']*100:5.2f}%")
        print(f"   Sharpe Ratio: {cartera['sharpe_ratio']:6.3f}")
        print("   Composición:")
        
        for activo, peso in cartera['weights'].items():
            if peso > 0.01:
                print(f"     {activo:8s}: {peso*100:5.1f}%")
    
    return cartera_sharpe, cartera_min_vol

def calcular_retornos_cartera(df, weights_dict):
    """Calcula retornos históricos de una cartera"""
    portfolio_returns = []
    
    for i in range(1, len(df)):
        day_return = 0
        valid_weights_sum = 0
        
        for activo, peso in weights_dict.items():
            if activo in df.columns:
                if pd.notna(df.iloc[i][activo]) and pd.notna(df.iloc[i-1][activo]):
                    ret_activo = (df.iloc[i][activo] / df.iloc[i-1][activo]) - 1
                    day_return += peso * ret_activo
                    valid_weights_sum += peso
        
        # Normalizar por pesos válidos
        if valid_weights_sum > 0:
            day_return = day_return / valid_weights_sum
        
        portfolio_returns.append(day_return)
    
    return portfolio_returns

def analizar_carteras_optimizadas(df, cartera_sharpe, cartera_vol, activos):
    """Analiza las carteras optimizadas con métricas avanzadas integrado en optimización"""
    
    if not cartera_sharpe and not cartera_vol:
        return pd.DataFrame()
    
    calculator = RiskCalculator()
    # Usar tasa dinámica del BCRA
    calculator.set_dynamic_risk_free_rate(df)
    resultados_carteras = []
    
    # Analizar cartera de máximo Sharpe
    if cartera_sharpe:
        portfolio_returns = calcular_retornos_cartera(df, cartera_sharpe['weights'])
        if len(portfolio_returns) > 30:
            metrics = calculator.calculate_metrics(pd.Series(portfolio_returns), df, "Cartera_Sharpe")
            if metrics:
                metrics['activo'] = 'Cartera Máximo Sharpe'
                metrics['tipo'] = 'Optimizada'
                resultados_carteras.append(metrics)
    
    # Analizar cartera de mínimo riesgo
    if cartera_vol:
        portfolio_returns = calcular_retornos_cartera(df, cartera_vol['weights'])
        if len(portfolio_returns) > 30:
            metrics = calculator.calculate_metrics(pd.Series(portfolio_returns), df, "Cartera_MinVol")
            if metrics:
                metrics['activo'] = 'Cartera Mínimo Riesgo'
                metrics['tipo'] = 'Optimizada'
                resultados_carteras.append(metrics)
    
    # Cartera Equal Weight para comparación
    equal_weights = {activo: 1/len(activos) for activo in activos}
    portfolio_returns = calcular_retornos_cartera(df, equal_weights)
    if len(portfolio_returns) > 30:
        metrics = calculator.calculate_metrics(pd.Series(portfolio_returns), df, "Cartera_Equal")
        if metrics:
            metrics['activo'] = 'Cartera Equal Weight'
            metrics['tipo'] = 'Benchmark'
            resultados_carteras.append(metrics)
    
    if resultados_carteras:
        df_carteras = pd.DataFrame(resultados_carteras)
        
        # Formatear para display
        df_display = df_carteras.copy()
        
        # Formatear porcentajes
        percentage_cols = ['rendimiento_anual', 'volatilidad', 'max_drawdown', 'alpha', 'tracking_error']
        for col in percentage_cols:
            if col in df_display.columns:
                df_display[col] = (df_display[col] * 100).round(2)
        
        # Formatear ratios
        ratio_cols = ['sharpe_ratio', 'sortino_ratio', 'treynor_ratio', 'information_ratio', 
                     'calmar_ratio', 'kelly_criterion', 'beta', 'r_squared']
        for col in ratio_cols:
            if col in df_display.columns:
                df_display[col] = df_display[col].round(3)
        
        print(f"\n📈 ANÁLISIS DE CARTERAS OPTIMIZADAS:")
        print("="*60)
        
        # Mostrar métricas principales
        main_cols = ['activo', 'rendimiento_anual', 'volatilidad', 'sharpe_ratio', 'max_drawdown']
        available_main = [col for col in main_cols if col in df_display.columns]
        if available_main:
            from IPython.display import display
            display(df_display[available_main])
        
        # Mostrar métricas avanzadas si están disponibles
        advanced_cols = ['activo', 'alpha', 'beta', 'treynor_ratio', 'kelly_criterion']
        available_advanced = [col for col in advanced_cols if col in df_display.columns]
        has_advanced_data = any(df_display[col].notna().any() for col in available_advanced[1:] if col in df_display.columns)
        
        if available_advanced and has_advanced_data:
            print(f"\n🔸 MÉTRICAS AVANZADAS:")
            display(df_display[available_advanced])
        
        return df_carteras
    
    return pd.DataFrame()

# Ejecutar optimización
if not returns_data.empty:
    print("🎯 Optimizando carteras...")
    cartera_opt_sharpe, cartera_opt_vol = optimizar_carteras(returns_data)
    
    # Análisis avanzado de carteras optimizadas integrado
    if cartera_opt_sharpe or cartera_opt_vol:
        metricas_carteras = analizar_carteras_optimizadas(df_data_completo, cartera_opt_sharpe, cartera_opt_vol, activos_reales)
        
        # Solo mantener métricas de carteras ya calculadas sin exportar por separado
        if not metricas_carteras.empty:
            print(f"\n📊 Métricas de carteras calculadas: {len(metricas_carteras)} carteras analizadas")
    
else:
    print("❌ No hay datos de retornos para optimización")
    cartera_opt_sharpe = cartera_opt_vol = None

🎯 Optimizando carteras...
✅ Usando tasa libre de riesgo REAL del BCRA: 25.46% anual
🎯 Carteras Optimizadas:
────────────────────────────────────────────────────────────

📊 Máximo Sharpe:
   Rendimiento: 52.25%
   Volatilidad: 29.43%
   Sharpe Ratio:  0.910
   Composición:
     EWZ     : 100.0%

📊 Mínimo Riesgo:
   Rendimiento: 24.84%
   Volatilidad: 22.93%
   Sharpe Ratio: -0.027
   Composición:
     SPY     :  35.6%
     EWZ     :  27.5%
     BHIL    :   3.5%
     IBM     :  33.3%
✅ Usando tasa libre de riesgo REAL del BCRA: 32.87% anual

📈 ANÁLISIS DE CARTERAS OPTIMIZADAS:

📈 ANÁLISIS DE CARTERAS OPTIMIZADAS:


,activo,rendimiento_anual,volatilidad,sharpe_ratio,max_drawdown
0,Cartera Máximo Sharpe,61.8400,27.5800,1.0510,-16.7000
1,Cartera Mínimo Riesgo,44.9800,22.7300,0.5330,-14.6000
2,Cartera Equal Weight,14.6900,30.8300,-0.5900,-23.3800



🔸 MÉTRICAS AVANZADAS:


,activo,alpha,beta,treynor_ratio,kelly_criterion
0,Cartera Máximo Sharpe,49.3800,0.1060,2.7410,0.0610
1,Cartera Mínimo Riesgo,40.9100,0.0320,3.8050,0.1580
2,Cartera Equal Weight,20.8200,0.0720,-2.5320,0.0320



📊 Métricas de carteras calculadas: 3 carteras analizadas


## 5. Backtesting y Validación de Estrategias

Validación histórica de las carteras optimizadas.

In [177]:
def backtest_estrategia(df, weights_dict, nombre):
    """Ejecuta backtesting de una estrategia"""
    
    # Calcular retornos de cartera
    portfolio_returns = []
    fechas = []
    
    for i in range(1, len(df)):
        fecha = df.iloc[i]['Fecha']
        port_return = 0
        
        for activo, peso in weights_dict.items():
            if activo in df.columns:
                if pd.notna(df.iloc[i][activo]) and pd.notna(df.iloc[i-1][activo]):
                    ret_activo = (df.iloc[i][activo] / df.iloc[i-1][activo]) - 1
                    port_return += peso * ret_activo
        
        portfolio_returns.append(port_return)
        fechas.append(fecha)
    
    # Calcular métricas
    port_returns = pd.Series(portfolio_returns)
    valor_acumulado = (1 + port_returns).cumprod() * 100
    
    ret_anual = port_returns.mean() * 252
    vol_anual = port_returns.std() * np.sqrt(252)
    sharpe = (ret_anual - 0.02) / vol_anual if vol_anual > 0 else 0
    
    # Maximum Drawdown
    running_max = valor_acumulado.cummax()
    drawdown = (valor_acumulado - running_max) / running_max
    max_dd = drawdown.min()
    
    return {
        'nombre': nombre,
        'fechas': fechas,
        'valores': valor_acumulado.tolist(),
        'rendimiento_anual': ret_anual,
        'volatilidad_anual': vol_anual,
        'sharpe_ratio': sharpe,
        'max_drawdown': max_dd
    }

def ejecutar_backtesting(df, cartera_sharpe, cartera_vol, activos):
    """Ejecuta backtesting de múltiples estrategias"""
    
    estrategias = []
    
    # Estrategia 1: Máximo Sharpe
    if cartera_sharpe:
        bt_sharpe = backtest_estrategia(df, cartera_sharpe['weights'], 'Máximo Sharpe')
        estrategias.append(bt_sharpe)
    
    # Estrategia 2: Mínimo Riesgo
    if cartera_vol:
        bt_vol = backtest_estrategia(df, cartera_vol['weights'], 'Mínimo Riesgo')
        estrategias.append(bt_vol)
    
    # Estrategia 3: Equal Weight
    equal_weights = {activo: 1/len(activos) for activo in activos}
    bt_equal = backtest_estrategia(df, equal_weights, 'Equal Weight')
    estrategias.append(bt_equal)
    
    # Crear DataFrame de resultados para display
    resultados_df = pd.DataFrame([
        {
            'Estrategia': est['nombre'],
            'Rendimiento (%)': f"{est['rendimiento_anual']*100:.2f}",
            'Volatilidad (%)': f"{est['volatilidad_anual']*100:.2f}",
            'Sharpe Ratio': f"{est['sharpe_ratio']:.3f}",
            'Max Drawdown (%)': f"{est['max_drawdown']*100:.2f}"
        }
        for est in estrategias
    ])
    
    print("📊 Resultados de Backtesting:")
    print("─" * 70)
    
    # Usar display() para mejor visualización
    from IPython.display import display
    display(resultados_df)
    
    # Gráfico comparativo
    fig = go.Figure()
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    
    for i, est in enumerate(estrategias):
        fig.add_trace(go.Scatter(
            x=est['fechas'],
            y=est['valores'],
            mode='lines',
            name=est['nombre'],
            line=dict(width=2, color=colors[i % len(colors)]),
            hovertemplate=f"{est['nombre']}: %{{y:.1f}}<extra></extra>"
        ))
    
    fig.update_layout(
        title='Comparación de Performance - Backtesting Histórico',
        xaxis_title='Fecha',
        yaxis_title='Valor de Cartera (Base 100)',
        template='plotly_white',
        height=500,
        hovermode='x unified'
    )
    
    # Mostrar y guardar como imagen estática
    fig.show()
    fig.write_image("backtesting_comparison.png", width=1000, height=500, scale=2)
    
    return estrategias

# Ejecutar backtesting
if cartera_opt_sharpe or cartera_opt_vol:
    print("🔄 Ejecutando backtesting...")
    resultados_bt = ejecutar_backtesting(df_data, cartera_opt_sharpe, cartera_opt_vol, activos)
else:
    print("❌ No hay carteras optimizadas para backtesting")
    resultados_bt = []

🔄 Ejecutando backtesting...
📊 Resultados de Backtesting:
──────────────────────────────────────────────────────────────────────
📊 Resultados de Backtesting:
──────────────────────────────────────────────────────────────────────


,Estrategia,Rendimiento (%),Volatilidad (%),Sharpe Ratio,Max Drawdown (%)
0,Máximo Sharpe,61.84,27.58,2.170,-16.70
1,Mínimo Riesgo,35.99,21.07,1.613,-13.37
2,Equal Weight,6.05,29.77,0.136,-22.97


## 6. Análisis de Escenarios y Stress Testing

Simulaciones Monte Carlo y análisis de escenarios adversos.

In [178]:
class MonteCarloAdvanced:
    """Simulador Monte Carlo avanzado para análisis de cartera"""
    
    def __init__(self, risk_free_rate=0.02):
        self.rf_rate = risk_free_rate
        np.random.seed(42)  # Para reproducibilidad
    
    def calculate_portfolio_metrics(self, weights_dict, returns_data):
        """Calcula métricas históricas de la cartera"""
        portfolio_returns = []
        
        for i in range(len(returns_data)):
            day_return = 0
            valid_weights_sum = 0
            
            for activo, peso in weights_dict.items():
                if activo in returns_data.columns:
                    if pd.notna(returns_data.iloc[i][activo]):
                        day_return += peso * returns_data.iloc[i][activo]
                        valid_weights_sum += peso
            
            # Normalizar por pesos válidos
            if valid_weights_sum > 0:
                day_return = day_return / valid_weights_sum
            
            portfolio_returns.append(day_return)
        
        portfolio_returns = np.array(portfolio_returns)
        portfolio_returns = portfolio_returns[~np.isnan(portfolio_returns)]
        
        return {
            'returns': portfolio_returns,
            'mean_daily': np.mean(portfolio_returns),
            'std_daily': np.std(portfolio_returns),
            'skewness': stats.skew(portfolio_returns),
            'kurtosis': stats.kurtosis(portfolio_returns),
            'sharpe_historical': (np.mean(portfolio_returns) * 252 - self.rf_rate) / (np.std(portfolio_returns) * np.sqrt(252))
        }
    
    def monte_carlo_simulation(self, weights_dict, returns_data, num_sims=5000, time_horizons=[21, 63, 126, 252]):
        """Simulación Monte Carlo avanzada con múltiples horizontes temporales"""
        
        print("🎲 Ejecutando simulación Monte Carlo avanzada...")
        
        # Calcular métricas históricas
        portfolio_metrics = self.calculate_portfolio_metrics(weights_dict, returns_data)
        mean_ret = portfolio_metrics['mean_daily']
        std_ret = portfolio_metrics['std_daily']
        
        results = {}
        all_paths = {}
        
        for days in time_horizons:
            print(f"   📊 Simulando {days} días ({days//21} meses aprox)...")
            
            # Generar caminos
            random_matrix = np.random.normal(mean_ret, std_ret, (num_sims, days))
            
            # Aplicar corrección por sesgo si hay asimetría
            if abs(portfolio_metrics['skewness']) > 0.5:
                skew_correction = np.random.gamma(2, 0.5, (num_sims, days)) - 1
                random_matrix += skew_correction * portfolio_metrics['skewness'] * 0.1
            
            # Calcular valores finales
            cumulative_returns = np.cumprod(1 + random_matrix, axis=1)
            final_values = cumulative_returns[:, -1]
            
            # Calcular retornos anualizados
            annual_returns = (final_values ** (252/days)) - 1
            
            # Estadísticas detalladas
            percentiles = [1, 5, 10, 25, 50, 75, 90, 95, 99]
            return_percentiles = np.percentile(annual_returns, percentiles)
            
            # Probabilidades de pérdida
            prob_loss = np.sum(annual_returns < 0) / len(annual_returns)
            prob_loss_10 = np.sum(annual_returns < -0.10) / len(annual_returns)
            prob_loss_20 = np.sum(annual_returns < -0.20) / len(annual_returns)
            prob_loss_30 = np.sum(annual_returns < -0.30) / len(annual_returns)
            
            # VaR y CVaR
            var_95 = np.percentile(annual_returns, 5)
            var_99 = np.percentile(annual_returns, 1)
            cvar_95 = np.mean(annual_returns[annual_returns <= var_95])
            cvar_99 = np.mean(annual_returns[annual_returns <= var_99])
            
            results[days] = {
                'horizon_months': days // 21,
                'percentiles': dict(zip(percentiles, return_percentiles)),
                'probabilities': {
                    'loss': prob_loss,
                    'loss_10': prob_loss_10,
                    'loss_20': prob_loss_20,
                    'loss_30': prob_loss_30
                },
                'risk_metrics': {
                    'var_95': var_95,
                    'var_99': var_99,
                    'cvar_95': cvar_95,
                    'cvar_99': cvar_99
                },
                'statistics': {
                    'mean': np.mean(annual_returns),
                    'std': np.std(annual_returns),
                    'min': np.min(annual_returns),
                    'max': np.max(annual_returns),
                    'sharpe': (np.mean(annual_returns) - self.rf_rate) / np.std(annual_returns)
                }
            }
            
            # Guardar algunos caminos para visualización
            if days == 252:  # Solo para 1 año
                all_paths[days] = cumulative_returns[:100]  # Primeros 100 caminos
        
        return results, all_paths, portfolio_metrics
    
    def stress_testing_advanced(self, weights_dict):
        """Stress testing avanzado con múltiples escenarios"""
        
        scenarios = {
            'Crisis 2008': {'shock': -0.45, 'duration': 180, 'recovery': 0.8},
            'COVID-19': {'shock': -0.35, 'duration': 60, 'recovery': 1.2},
            'Crisis Financiera': {'shock': -0.30, 'duration': 120, 'recovery': 0.9},
            'Recesión': {'shock': -0.20, 'duration': 90, 'recovery': 1.0},
            'Corrección Técnica': {'shock': -0.15, 'duration': 30, 'recovery': 1.1},
            'Inflación Alta': {'shock': -0.10, 'duration': 252, 'recovery': 0.7},
            'Guerra/Geopolítica': {'shock': -0.25, 'duration': 45, 'recovery': 0.9}
        }
        
        results = {}
        
        for scenario_name, params in scenarios.items():
            portfolio_impact = 0
            
            for activo, peso in weights_dict.items():
                # Clasificar activos por tipo para ajustar shocks
                if activo in ['SPY', 'EWZ', 'QQQ']:
                    # ETFs internacionales - siguen mercados globales
                    asset_shock = params['shock'] * 1.0
                elif activo in ['AAPL', 'MSFT', 'GOOGL', 'IBM']:
                    # Tech stocks - más volátiles
                    if scenario_name == 'COVID-19':
                        asset_shock = params['shock'] * 0.8  # Tech resiliente en COVID
                    else:
                        asset_shock = params['shock'] * 1.3
                elif activo in ['GGAL', 'PAMP', 'TXAR', 'BBAR']:
                    # Activos argentinos - correlación específica
                    if scenario_name == 'Guerra/Geopolítica':
                        asset_shock = params['shock'] * 1.5  # Países emergentes más afectados
                    else:
                        asset_shock = params['shock'] * 0.9
                elif activo in ['GOLD', 'GLD']:
                    # Oro - cobertura en crisis
                    asset_shock = params['shock'] * -0.3  # Oro sube en crisis
                else:
                    # Otros activos
                    asset_shock = params['shock'] * 0.85
                
                portfolio_impact += peso * asset_shock
            
            # Calcular tiempo de recuperación esperado
            recovery_time = params['duration'] * (1 / params['recovery'])
            
            results[scenario_name] = {
                'impact': portfolio_impact,
                'duration_days': params['duration'],
                'recovery_time': recovery_time,
                'severity': abs(portfolio_impact)
            }
        
        return results

def create_monte_carlo_visualizations(mc_results, paths, portfolio_metrics, stress_results):
    """Crea visualizaciones avanzadas para Monte Carlo"""
    
    # 1. Distribución de retornos para diferentes horizontes
    fig_dist = make_subplots(
        rows=2, cols=2,
        subplot_titles=['1 Mes', '3 Meses', '6 Meses', '1 Año'],
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    horizons = [21, 63, 126, 252]
    positions = [(1,1), (1,2), (2,1), (2,2)]
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
    
    for i, (days, pos, color) in enumerate(zip(horizons, positions, colors)):
        if days in mc_results:
            # Crear distribución sintética para visualización
            mean_ret = mc_results[days]['statistics']['mean']
            std_ret = mc_results[days]['statistics']['std']
            x_vals = np.linspace(mean_ret - 4*std_ret, mean_ret + 4*std_ret, 100)
            y_vals = stats.norm.pdf(x_vals, mean_ret, std_ret)
            
            fig_dist.add_trace(
                go.Scatter(x=x_vals*100, y=y_vals, 
                          fill='tozeroy', fillcolor=color, opacity=0.7,
                          name=f'{days//21} Meses', line=dict(color=color, width=2)),
                row=pos[0], col=pos[1]
            )
            
            # Añadir líneas de percentiles importantes
            p5 = mc_results[days]['percentiles'][5] * 100
            p95 = mc_results[days]['percentiles'][95] * 100
            
            fig_dist.add_vline(x=p5, line_dash="dash", line_color="red", 
                              row=pos[0], col=pos[1])
            fig_dist.add_vline(x=p95, line_dash="dash", line_color="green", 
                              row=pos[0], col=pos[1])
    
    fig_dist.update_layout(
        title='Distribución de Retornos Anualizados por Horizonte Temporal',
        height=600, showlegend=False, template='plotly_white'
    )
    fig_dist.update_xaxes(title_text="Retorno Anualizado (%)")
    fig_dist.update_yaxes(title_text="Densidad de Probabilidad")
    fig_dist.show()
    fig_dist.write_image("monte_carlo_distribuciones.png", width=1200, height=600, scale=2)
    
    # 2. Caminos de simulación (solo para 1 año)
    if 252 in paths:
        fig_paths = go.Figure()
        
        # Mostrar algunos caminos individuales
        for i in range(min(50, len(paths[252]))):
            fig_paths.add_trace(go.Scatter(
                x=list(range(252)),
                y=(paths[252][i] - 1) * 100,
                mode='lines',
                line=dict(width=0.5, color='lightblue'),
                opacity=0.3,
                showlegend=False,
                hovertemplate='Día %{x}<br>Retorno: %{y:.1f}%<extra></extra>'
            ))
        
        # Añadir percentiles
        percentiles_paths = np.percentile(paths[252], [5, 50, 95], axis=0)
        
        fig_paths.add_trace(go.Scatter(
            x=list(range(252)),
            y=(percentiles_paths[0] - 1) * 100,
            mode='lines',
            name='Percentil 5%',
            line=dict(color='red', width=3, dash='dash')
        ))
        
        fig_paths.add_trace(go.Scatter(
            x=list(range(252)),
            y=(percentiles_paths[1] - 1) * 100,
            mode='lines',
            name='Mediana',
            line=dict(color='blue', width=3)
        ))
        
        fig_paths.add_trace(go.Scatter(
            x=list(range(252)),
            y=(percentiles_paths[2] - 1) * 100,
            mode='lines',
            name='Percentil 95%',
            line=dict(color='green', width=3, dash='dash')
        ))
        
        fig_paths.update_layout(
            title='Caminos de Simulación Monte Carlo (1 Año)',
            xaxis_title='Días',
            yaxis_title='Retorno Acumulado (%)',
            template='plotly_white',
            height=500,
            hovermode='x unified'
        )
        fig_paths.show()
        fig_paths.write_image("monte_carlo_caminos.png", width=1000, height=500, scale=2)
    
    # 3. Mapa de calor de probabilidades de pérdida
    fig_heatmap = go.Figure()
    
    horizons = [21, 63, 126, 252]
    horizon_labels = ['1 Mes', '3 Meses', '6 Meses', '1 Año']
    loss_levels = ['Cualquier pérdida', 'Pérdida >10%', 'Pérdida >20%', 'Pérdida >30%']
    loss_keys = ['loss', 'loss_10', 'loss_20', 'loss_30']
    
    heatmap_data = []
    for days in horizons:
        if days in mc_results:
            row_data = []
            for key in loss_keys:
                row_data.append(mc_results[days]['probabilities'][key] * 100)
            heatmap_data.append(row_data)
    
    fig_heatmap.add_trace(go.Heatmap(
        z=heatmap_data,
        x=loss_levels,
        y=horizon_labels,
        colorscale='Reds',
        text=[[f'{val:.1f}%' for val in row] for row in heatmap_data],
        texttemplate='%{text}',
        hovertemplate='Horizonte: %{y}<br>Tipo: %{x}<br>Probabilidad: %{z:.1f}%<extra></extra>'
    ))
    
    fig_heatmap.update_layout(
        title='Mapa de Probabilidades de Pérdida',
        xaxis_title='Tipo de Pérdida',
        yaxis_title='Horizonte Temporal',
        template='plotly_white',
        height=400
    )
    fig_heatmap.show()
    fig_heatmap.write_image("monte_carlo_heatmap.png", width=800, height=400, scale=2)
    
    # 4. Stress Testing Visualización
    if stress_results:
        scenarios = list(stress_results.keys())
        impacts = [stress_results[s]['impact'] * 100 for s in scenarios]
        colors_stress = ['red' if imp < -20 else 'orange' if imp < -10 else 'yellow' for imp in impacts]
        
        fig_stress = go.Figure(go.Bar(
            x=impacts,
            y=scenarios,
            orientation='h',
            marker_color=colors_stress,
            text=[f'{imp:+.1f}%' for imp in impacts],
            textposition='auto',
            hovertemplate='%{y}<br>Impacto: %{x:+.1f}%<extra></extra>'
        ))
        
        fig_stress.update_layout(
            title='Análisis de Stress Testing - Impacto en Cartera',
            xaxis_title='Impacto en Cartera (%)',
            yaxis_title='Escenario de Stress',
            template='plotly_white',
            height=500
        )
        fig_stress.add_vline(x=0, line_dash="dash", line_color="black")
        fig_stress.show()
        fig_stress.write_image("stress_testing.png", width=1000, height=500, scale=2)

def analizar_escenarios_avanzado(cartera_sharpe, returns_data):
    """Ejecuta análisis completo de escenarios con Monte Carlo avanzado"""
    
    if not cartera_sharpe or returns_data.empty:
        print("❌ No hay datos suficientes para análisis de escenarios")
        return None, None, None
    
    print("🚀 ANÁLISIS AVANZADO DE ESCENARIOS")
    print("=" * 60)
    
    # Inicializar simulador
    mc_simulator = MonteCarloAdvanced()
    
    # Ejecutar Monte Carlo
    mc_results, paths, portfolio_metrics = mc_simulator.monte_carlo_simulation(
        cartera_sharpe['weights'], returns_data, num_sims=5000
    )
    
    # Ejecutar Stress Testing
    stress_results = mc_simulator.stress_testing_advanced(cartera_sharpe['weights'])
    
    # Mostrar resultados resumidos
    print(f"\n📊 RESULTADOS MONTE CARLO:")
    print("─" * 40)
    
    for days, results in mc_results.items():
        months = days // 21
        print(f"\n🎯 Horizonte: {months} mes{'es' if months > 1 else ''} ({days} días)")
        print(f"   Retorno esperado: {results['statistics']['mean']*100:+6.1f}%")
        print(f"   Volatilidad: {results['statistics']['std']*100:6.1f}%")
        print(f"   Sharpe Ratio: {results['statistics']['sharpe']:6.3f}")
        print(f"   VaR 95%: {results['risk_metrics']['var_95']*100:+6.1f}%")
        print(f"   CVaR 95%: {results['risk_metrics']['cvar_95']*100:+6.1f}%")
        print(f"   Prob. pérdida: {results['probabilities']['loss']*100:5.1f}%")
    
    print(f"\n⚡ STRESS TESTING:")
    print("─" * 40)
    
    # Ordenar por severidad
    sorted_stress = sorted(stress_results.items(), 
                          key=lambda x: x[1]['severity'], reverse=True)
    
    for scenario, data in sorted_stress[:5]:  # Top 5 más severos
        print(f"   {scenario:20s}: {data['impact']*100:+6.1f}% "
              f"(Duración: {data['duration_days']:3d} días)")
    
    # Crear visualizaciones
    print(f"\n📈 Generando visualizaciones avanzadas...")
    create_monte_carlo_visualizations(mc_results, paths, portfolio_metrics, stress_results)
    
    return mc_results, stress_results, portfolio_metrics

# Ejecutar análisis avanzado
if cartera_opt_sharpe and not returns_data.empty:
    print("🎯 Ejecutando análisis avanzado de escenarios...")
    mc_results, stress_results, portfolio_metrics = analizar_escenarios_avanzado(cartera_opt_sharpe, returns_data)
    
    # Solo mostrar análisis sin exportación adicional
    print(f"\n📊 Análisis Monte Carlo completado para {len(mc_results)} horizontes temporales")
        
else:
    print("❌ No hay cartera optimizada para análisis de escenarios")
    mc_results = stress_results = portfolio_metrics = None

🎯 Ejecutando análisis avanzado de escenarios...
🚀 ANÁLISIS AVANZADO DE ESCENARIOS
🎲 Ejecutando simulación Monte Carlo avanzada...
   📊 Simulando 21 días (1 meses aprox)...
   📊 Simulando 63 días (3 meses aprox)...
   📊 Simulando 126 días (6 meses aprox)...
   📊 Simulando 252 días (12 meses aprox)...

📊 RESULTADOS MONTE CARLO:
────────────────────────────────────────

🎯 Horizonte: 1 mes (21 días)
   Retorno esperado: +220.1%
   Volatilidad:  413.5%
   Sharpe Ratio:  0.527
   VaR 95%:  -64.7%
   CVaR 95%:  -75.4%
   Prob. pérdida:  26.7%

🎯 Horizonte: 3 meses (63 días)
   Retorno esperado: +123.1%
   Volatilidad:  139.5%
   Sharpe Ratio:  0.868
   VaR 95%:  -29.5%
   CVaR 95%:  -43.0%
   Prob. pérdida:  14.0%

🎯 Horizonte: 6 meses (126 días)
   Retorno esperado: +103.8%
   Volatilidad:   87.2%
   Sharpe Ratio:  1.167
   VaR 95%:   -5.3%
   CVaR 95%:  -19.4%
   Prob. pérdida:   6.5%

🎯 Horizonte: 12 meses (252 días)
   Retorno esperado:  +97.4%
   Volatilidad:   58.4%
   Sharpe Ratio:  1.


📊 Análisis Monte Carlo completado para 4 horizontes temporales


## 7. Reporte Ejecutivo y Recomendaciones

Resumen consolidado con recomendaciones estratégicas para la toma de decisiones.

In [179]:
def generar_reporte_ejecutivo():
    """Genera reporte ejecutivo consolidado"""
    
    print("=" * 80)
    print("📋 REPORTE EJECUTIVO - ANÁLISIS DE CARTERA PROFESIONAL")
    print("=" * 80)
    
    # 1. Resumen del análisis
    total_activos = len(df_metricas) if 'df_metricas' in locals() and not df_metricas.empty else 0
    fecha_inicio = df_data['Fecha'].min().strftime('%d/%m/%Y') if not df_data.empty else "N/A"
    fecha_fin = df_data['Fecha'].max().strftime('%d/%m/%Y') if not df_data.empty else "N/A"
    
    print(f"\n📊 RESUMEN DEL ANÁLISIS:")
    print(f"   • Activos analizados: {total_activos}")
    print(f"   • Período: {fecha_inicio} → {fecha_fin}")
    print(f"   • Total observaciones: {len(df_data):,}")
    
    # 2. Performance individual
    if 'df_metricas' in locals() and not df_metricas.empty:
        print(f"\n📈 TOP PERFORMERS:")
        
        # Mejor Sharpe
        mejor_sharpe = df_metricas.loc[df_metricas['sharpe_ratio'].idxmax()]
        print(f"   🏆 Mejor Sharpe Ratio: {mejor_sharpe['activo']} ({mejor_sharpe['sharpe_ratio']:.3f})")
        
        # Mayor rendimiento
        mayor_ret = df_metricas.loc[df_metricas['rendimiento_anual'].idxmax()]
        print(f"   📈 Mayor Rendimiento: {mayor_ret['activo']} ({mayor_ret['rendimiento_anual']*100:+.1f}%)")
        
        # Menor volatilidad
        menor_vol = df_metricas.loc[df_metricas['volatilidad'].idxmin()]
        print(f"   🛡️  Menor Riesgo: {menor_vol['activo']} ({menor_vol['volatilidad']*100:.1f}%)")
    
    # 3. Carteras optimizadas
    print(f"\n🎯 CARTERAS RECOMENDADAS:")
    
    if 'cartera_opt_sharpe' in locals() and cartera_opt_sharpe:
        print(f"   🏆 Cartera Óptima (Máximo Sharpe):")
        print(f"      Rendimiento esperado: {cartera_opt_sharpe['rendimiento']*100:5.1f}%")
        print(f"      Volatilidad esperada: {cartera_opt_sharpe['volatilidad']*100:5.1f}%")
        print(f"      Sharpe Ratio: {cartera_opt_sharpe['sharpe_ratio']:5.3f}")
        
        print("      Composición:")
        for activo, peso in cartera_opt_sharpe['weights'].items():
            if peso > 0.02:
                print(f"         {activo}: {peso*100:4.1f}%")
    
    # 4. Resultados de backtesting
    if 'resultados_bt' in locals() and resultados_bt:
        print(f"\n🔄 VALIDACIÓN HISTÓRICA:")
        mejor_bt = max(resultados_bt, key=lambda x: x['sharpe_ratio'])
        print(f"   Mejor estrategia histórica: {mejor_bt['nombre']}")
        print(f"   Rendimiento anual: {mejor_bt['rendimiento_anual']*100:+.1f}%")
        print(f"   Sharpe Ratio: {mejor_bt['sharpe_ratio']:.3f}")
        print(f"   Max Drawdown: {mejor_bt['max_drawdown']*100:.1f}%")
    
    # 5. Análisis de riesgos
    if 'mc_results' in locals() and mc_results:
        print(f"\n⚠️  ANÁLISIS DE RIESGOS (1 año):")
        print(f"   Escenario esperado: {mc_results['return_percentiles'][50]*100:+.1f}%")
        print(f"   Escenario pesimista: {mc_results['return_percentiles'][5]*100:+.1f}%")
        print(f"   Probabilidad de pérdida: {mc_results['prob_loss']*100:.1f}%")
    
    if 'stress_results' in locals() and stress_results:
        print(f"\n⚡ STRESS TESTING:")
        for scenario, impact in stress_results.items():
            print(f"   {scenario}: {impact*100:+.1f}%")
    
    # 6. Recomendaciones
    print(f"\n💡 RECOMENDACIONES ESTRATÉGICAS:")
    
    recomendaciones = [
        "Implementar cartera óptima con rebalanceo trimestral",
        "Establecer límites de VaR para control de riesgo",
        "Monitorear correlaciones durante alta volatilidad",
        "Considerar cobertura para escenarios de crisis",
        "Revisar composición semestralmente"
    ]
    
    if 'cartera_opt_sharpe' in locals() and cartera_opt_sharpe:
        if cartera_opt_sharpe['sharpe_ratio'] > 1.0:
            recomendaciones.insert(0, "✅ Cartera muestra excelente relación riesgo-rendimiento")
        elif cartera_opt_sharpe['sharpe_ratio'] < 0.5:
            recomendaciones.insert(0, "⚠️  Considerar diversificación adicional")
    
    for i, rec in enumerate(recomendaciones, 1):
        print(f"   {i}. {rec}")
    
    # 7. Próximos pasos
    print(f"\n🚀 PRÓXIMOS PASOS:")
    pasos = [
        "Configurar sistema de alertas por límites de riesgo",
        "Programar revisión mensual de métricas",
        "Desarrollar dashboard en tiempo real",
        "Evaluar incorporación de factores ESG"
    ]
    
    for i, paso in enumerate(pasos, 1):
        print(f"   {i}. {paso}")
    
    # Disclaimer
    print(f"\n⚖️  DISCLAIMER:")
    print(f"   Este análisis se basa en datos históricos y no garantiza")
    print(f"   resultados futuros. Consultar con asesor financiero.")
    
    print(f"\n🎉 Análisis completado - {datetime.now().strftime('%d/%m/%Y %H:%M')}")
    print("=" * 80)

# Generar reporte final
generar_reporte_ejecutivo()

print(f"\n🎯 ARCHIVOS EXCEL ACTUALIZADOS:")
print(f"   📊 Datos_historicos_de_la_cartera.xlsx - Hoja1 ACTUALIZADA con datos completos")
print(f"   📈 Ratios Managment financiero cartera.xlsx - TODAS LAS HOJAS actualizadas con cálculos:")
print(f"       • Drawdown - ACTUALIZADA con Max Drawdown, Ulcer Index y Calmar Ratio")
print(f"       • VaR - Actualizada con VaR y CVaR calculados")
print(f"       • VaR cond - Actualizada con CVaR")
print(f"       • Kelly criterion - Actualizada con Kelly Criterion calculado")
print(f"       • Sharpe - Actualizada con Sharpe Ratio y métricas de rendimiento")
print(f"       • Sortino - Actualizada con Sortino Ratio y drawdown")
print(f"       • Alpha beta - Actualizada con Alpha, Beta, R² y Treynor Ratio")
print(f"\n✅ ACTUALIZACIÓN COMPLETA - Los archivos Excel existentes fueron actualizados con todos los cálculos")

📋 REPORTE EJECUTIVO - ANÁLISIS DE CARTERA PROFESIONAL

📊 RESUMEN DEL ANÁLISIS:
   • Activos analizados: 0
   • Período: 03/01/2025 → 02/09/2025
   • Total observaciones: 172

🎯 CARTERAS RECOMENDADAS:

💡 RECOMENDACIONES ESTRATÉGICAS:
   1. Implementar cartera óptima con rebalanceo trimestral
   2. Establecer límites de VaR para control de riesgo
   3. Monitorear correlaciones durante alta volatilidad
   4. Considerar cobertura para escenarios de crisis
   5. Revisar composición semestralmente

🚀 PRÓXIMOS PASOS:
   1. Configurar sistema de alertas por límites de riesgo
   2. Programar revisión mensual de métricas
   3. Desarrollar dashboard en tiempo real
   4. Evaluar incorporación de factores ESG

⚖️  DISCLAIMER:
   Este análisis se basa en datos históricos y no garantiza
   resultados futuros. Consultar con asesor financiero.

🎉 Análisis completado - 07/09/2025 17:42

🎯 ARCHIVOS EXCEL ACTUALIZADOS:
   📊 Datos_historicos_de_la_cartera.xlsx - Hoja1 ACTUALIZADA con datos completos
   📈 R

# Análisis Profesional de Cartera de Inversiones

**Análisis cuantitativo integral de riesgo y rendimiento**

---

## Resumen Ejecutivo

Este análisis combina datos históricos de Excel con información actualizada de APIs financieras para realizar un estudio completo de la cartera de inversiones, incluyendo:

- **Métricas de Riesgo**: VaR, CVaR, Maximum Drawdown
- **Indicadores de Rendimiento**: Sharpe Ratio, Sortino Ratio
- **Optimización de Cartera**: Frontera eficiente y carteras óptimas
- **Backtesting**: Validación histórica de estrategias
- **Análisis de Escenarios**: Stress testing y simulaciones Monte Carlo

---

*Análisis realizado el: 7 de Septiembre, 2025*